# Geo data example

This example uses the Smart Citizen API python wrapper to make an animation of the SCK history in the world. It uses folium (a leaflet.js implementation)

In [ ]:
from src.data.api import ScApiDevice
import folium
from folium import plugins
import pandas as pd
import branca

## Get data and clean it

Use the `ScApiDevice.get_word_map()` to get the whole map from the API

In [ ]:
# Sensor IDs
sensors = ScApiDevice.get_world_map(full = True)
sensors.set_index('kit_id')
sensors.head(5)

In [ ]:
def coordinates(x):
    return [x['longitude'], x['latitude']] # for whatever reason, leaflet uses the coordinates in reverse

def color(x):
    iSCAPE_IDs =[19, 20, 21, 28]
    making_sense_IDs = [11, 14]
    SCK_21_IDs = [26]
    
    if x['kit_id'] in iSCAPE_IDs: color = '#7dbd4c'
    elif x['kit_id'] in making_sense_IDs: color = '#f88027'
    elif x['kit_id'] in SCK_21_IDs: color = '#ffb500'
    else: color = '#0019ff'
        
    return color

def validate(x):
    
    if x['last_reading_at'] is None: return False
    if x['added_at'] is None: return False
    if x['coordinates'] == [None, None]: return False
    if x['date_list'] == []: return False
    
    return True

def range_list(x):
    
    date_r = pd.date_range(start=x['added_at'], end=x['last_reading_at'], normalize = True, freq='W').strftime('%Y-%m-%d')
    date_l = list()
    for item in date_r.values: date_l.append(str(item))

    return date_l

Now let's add some colors in depending on the version. We add the coordinates and some validation steps

In [ ]:
sensors['color'] = sensors.apply(lambda x: color(x), axis=1)
sensors['coordinates'] = sensors.apply(lambda x: coordinates(x), axis=1)
sensors['date_list'] = sensors.apply(lambda x: range_list(x), axis=1)
sensors['valid'] = sensors.apply(lambda x: validate(x), axis=1)

sensors = sensors[(sensors['valid'] == True)]

We put it all in folium format

In [ ]:
features = []

for sensor in sensors.index:
        features.append(
            {
                'type': 'Feature',
                'geometry': {
                    'type': 'LineString',
                    'coordinates': [sensors.loc[sensor, 'coordinates']]*len(sensors.loc[sensor, 'date_list']),
                    'popup': str(sensors.loc[sensor, 'kit_id']),
                },
                'properties': {
                    'times': sensors.loc[sensor, 'date_list'],
                    'icon': 'circle',
                    'iconstyle': {
                        'fillColor': sensors.loc[sensor, 'color'],
                        'fillOpacity': '0.6',
                        'stroke': 'false',
                        'radius': '6'
                    },
                    'style': {'weight': '0'},
                    'id': 'man'
                }
            }
        )

And display it. You can also save this to an html file

In [ ]:
m = folium.Map(
    location=[41.3947688,2.0787279],
    tiles='Stamen Toner',
    zoom_start=2.5,
)

plugins.TimestampedGeoJson(
    {
        'type': 'FeatureCollection',
        'features': features
    },
    period='P1W',
    add_last_point=True,
    auto_play=False,
    loop=False,
    max_speed=5,
    loop_button=True,
    date_options='YYYY/MM/DD',
    time_slider_drag_update=True,
    duration='P1W'
).add_to(m)

## Either save it or display it
# m.save(<html_save_path>)
m